# Output 2016 goals

depends on download_sheets

- 2016/goals.csv

In [ ]:
# output 2016 goals

import pandas as pd
import re
import os
from lib.vars import dirs
from lib.utils import load_csv,save_csv

raw=load_csv(f"{dirs().input.sheets.general}/H28/H28.csv")
raw


data=pd.DataFrame([])
data["layer1_id"]=raw["第1層（大項目）"].str.extract(r"^(.)")
data["layer1_item"]=raw["第1層（大項目）"].str.extract(r"^. ?(.+)")
data["layer2_id"]=raw["第2層（中項目）"].str.extract(r"^.\-(\d+)")
data["layer2_id"]=data["layer1_id"]+"-"+data["layer2_id"].str.zfill(2)
data["layer2_item"]=raw["第2層（中項目）"].str.extract(r"^.\-\d+ (.+)")
data["layer3_id"]=raw["第3層（小項目）"].str.extract(r"^.\-\d+\-(\d+)")
data["layer3_id"]=data["layer2_id"]+"-"+data["layer3_id"].str.zfill(2)
data["layer3_item"]=raw["第3層（小項目）"].str.extract(r"^.\-\d+\-\d+\) (.+)")
raw["layer3_id"]=data["layer3_id"]

layer4_id_list=[]
text4_list=[]
current_parent=""
prev_text=""
current_index=0
for index,row in raw.iterrows():
  text=row["第4層（細小項目）"]
  parent=row["layer3_id"]
  if parent!= current_parent:
    current_index=0
    prev_text=""
  if prev_text!= text:
    current_index=current_index+1
  current_parent=parent
  prev_text=text
  if text=="なし":
    layer4_id_list.append(f"{parent}-na")
    text4_list.append(text)
  else:
    layer4_id_list.append(f"{parent}-{str(current_index).zfill(2)}")
    text4_list.append(re.sub(r"^.\-\d+\-\d+\)\-\(\d+\) ","",str(text)))

data["layer4_id"]=layer4_id_list
data["layer4_item"]=text4_list
raw["layer4_id"]=data["layer4_id"]

layer5_id_list=[]
text5_list=[]
current_parent=""
prev_text=""
current_index=0
for index,row in raw.iterrows():
  text=row["第5層（学修目標）"]
  parent=row["layer4_id"]
  if parent!= current_parent:
    current_index=0
    prev_text=""
  if prev_text!= text:
    current_index=current_index+1
  current_parent=parent
  prev_text=text
  if text=="なし":
    layer5_id_list.append(f"{parent}-na")
    text5_list.append(text)
  else:
    layer5_id_list.append(f"{parent}-{str(current_index).zfill(2)}")
    item_text=re.sub(r"^([.０-９0-9]{1,2})( |\.|．)","",str(text))
    item_text=re.sub(r"^[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖]","",str(item_text))
    text5_list.append(item_text)

data["layer5_id"]=layer5_id_list
data["layer5_item"]=text5_list

os.makedirs(dirs().output.dir_2016,exist_ok=True)
save_csv(data,f"{dirs().output.dir_2016}/goals.csv")
print(f"output... goals.csv")
data